In [1]:
import json
import requests
import os
import pandas as pd
from neo4j import GraphDatabase
from data_normalizer import normalize
from dotenv import load_dotenv
import constants
import prettytable

load_dotenv()

C:\Users\ToanLe\AppData\Roaming\Python\Python37\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


False

In [2]:
def get_concepts(text):
    with open('data/intent_alias_data.json', encoding="utf8") as f:
        dictionary = json.load(f)

    out = {}
    for concept in dictionary:
        for alias in sorted(dictionary[concept], key=len, reverse=1):
            if alias in text:
                out[concept] = alias
                break
    return out

In [13]:
def get_match_relation(concept_key_list):
    with open('data/match_relation_dict.json', encoding="utf8") as f:
        match_relation_dict = json.load(f)

    match_relation_out = []
    for start_node in match_relation_dict:
        if start_node in concept_key_list:
            for end_node in match_relation_dict[start_node]:
                if end_node in concept_key_list:
                    match_relation_out.append(
                        match_relation_dict[start_node][end_node])

    return match_relation_out

In [4]:
NER_MODEL_BERT = "phobert_large"
NER_MODEL_BILSTM = "BiLSTM"
NER_MODEL_BILSTM_CRF = "BiLSTM+CRF"

INTENT_MODEL_ONE_VS_REST = "onevsrest"


def extract_ner(text, model=NER_MODEL_BERT):
    """
    Input Arguments:
        - text : the sentence which will be extracted NER
    """
    ner_service_url = os.getenv("NER_SERVICE_URL",
                                default="http://localhost:8001/api/v1/ner")

    data = {'model': model, 'text': text}

    headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}

    r = requests.post(ner_service_url, data=json.dumps(data), headers=headers)

    return r.json()

In [5]:
def ner_postprocess(entities_raw):
    """Postprocess for NER: 
    1. normalize values
    2. only accept the related information with previous question

    Args:
        entities_raw (Dict): result of NER service

    Returns:
        Dict, Dict: raw and normalized entities
    """
    entities_normed = dict()
    entities_raw_out = dict()

    for entity in entities_raw:
        key = entity['label']
        if key == 'O': continue

        value_raw = entity['content']
        if key not in entities_raw_out.keys():
            entities_raw_out[key] = [value_raw]
        else:
            entities_raw_out[key].append(value_raw)

        value_normed = normalize(value_raw, key)
        if key not in entities_normed.keys():
            entities_normed[key] = [value_normed]
        else:
            entities_normed[key].append(value_normed)

    return entities_raw_out, entities_normed

In [6]:
def get_entities(text):
    entities_raw = extract_ner(text)
    _, entities = ner_postprocess(entities_raw)
    return entities

In [34]:
##
# Remove the label which has a condition from returned_label.
#
# Argument:
#     matched_labels: The labels which were matched.
#     returned_labels: The labels which were returned.
#     attr: condition attribute.
# Return:
#     returned_labels.
# #
def recheck_returned_labels(matched_labels, returned_labels, attr):
    for match_label in matched_labels:
        if attr in match_label:
            returned_labels.remove(match_label)

    return returned_labels


In [35]:
def gen_query(concepts_keys, evidences_keys, targets, evidences,
              match_relation):
    out = """
    MATCH {} 
    WHERE {}
    RETURN {}
    LIMIT 50
        """
    conditions = []
    matched_labels = []  # Format: (alias:Node_name)
    condition_labels = []  # Format: alias.individual = 'value'
    returned_labels = []  # Format: (alias.individual)

    ##
    # Define matched labels.
    # #
    if not concepts_keys:
        matched_labels = ['n']
    else:
        for idx, target in enumerate(concepts_keys):
            matched_labels.append('(' + target + ':' + target.title() + ')')
    ##
    # Define returned labels.
    # #
    returned_labels = matched_labels.copy()

    ##
    # Define condition labels.
    # #
    attrs = [constants.LABEL_REAL_ESTATE_TYPE, constants.LABEL_REAL_ESTATE_SUB_TYPE, \
            constants.LABEL_POSITION, constants.LABEL_DIRECTION, \
            constants.LABEL_FRONT_LENGTH, constants.LABEL_ROAD_WIDTH, \
            constants.LABEL_FLOOR, constants.LABEL_BED_ROOM, constants.LABEL_LIVING_ROOM, constants.LABEL_BATH_ROOM, \
            constants.LABEL_SURROUNDING, constants.LABEL_PROJECT_NAME, \
            constants.LABEL_LEGAL, constants.LABEL_TRANSACTION]

    key2dbcol = {
        'tang': constants.LABEL_FLOOR,
        'ban cong': constants.LABEL_FLOOR_BAN_CONG,
        'gac': constants.LABEL_FLOOR_GAC,
        'ham': constants.LABEL_FLOOR_HAM,
        'lung': constants.LABEL_FLOOR_LUNG,
        'san thuong': constants.LABEL_FLOOR_SAN_THUONG,
        'tret': constants.LABEL_FLOOR_TRET
    }

    for attr in attrs:
        if attr in evidences:
            if attr == constants.LABEL_FLOOR:
                for val in evidences[attr]:
                    target_k = key2dbcol[val['type']]
                    target_v = val['value']

                    returned_labels = recheck_returned_labels(
                        matched_labels, returned_labels, target_k)

                    conditions.append(f"{target_k}.individual = '{target_v}'")
            else:
                returned_labels = recheck_returned_labels(
                    matched_labels, returned_labels, attr)

                if attr == constants.LABEL_REAL_ESTATE_TYPE or attr == constants.LABEL_REAL_ESTATE_SUB_TYPE:
                    conditions.append(
                        f"{attr}.individual IN {evidences[attr]}")
                else:
                    conditions.append(
                        f"{attr}.individual = '{evidences[attr][0]}'")

    ##
    # Condition for city, district, ward, street.
    # #
    loc_attrs = [
        constants.LABEL_DISTRICT, constants.LABEL_CITY, constants.LABEL_WARD,
        constants.LABEL_STREET
    ]

    for attr in loc_attrs:
        if attr in evidences:
            returned_labels = recheck_returned_labels(matched_labels,
                                                      returned_labels, attr)

            conditions.append(f"{attr}.individual = '{evidences[attr][0]}'")

    ##
    # Condition for price.
    # #
    PRICE_OFFSET_CONST = 0.1

    if constants.LABEL_PRICE in evidences:
        for ele in evidences[constants.LABEL_PRICE][:1]:
            low, high = ele

            if high is None:
                high = low + low * PRICE_OFFSET_CONST
                low = low - low * PRICE_OFFSET_CONST

            returned_labels = recheck_returned_labels(matched_labels,
                                                      returned_labels,
                                                      constants.LABEL_PRICE)

            conditions.append(
                f"'{low}' <= {constants.LABEL_PRICE}.individual <= '{high}'")

    ##
    # Condition for area.
    # #
    AREA_OFFSET_CONST = 0.1

    if constants.LABEL_AREA in evidences:
        for ele in evidences[constants.LABEL_AREA][:1]:
            low, high = ele

            if high is None:
                high = low + low * AREA_OFFSET_CONST
                low = low - low * AREA_OFFSET_CONST

            returned_labels = recheck_returned_labels(matched_labels,
                                                      returned_labels,
                                                      constants.LABEL_AREA)

            conditions.append(
                f"'{low}' <= {constants.LABEL_AREA}.individual <= '{high}'")

    ##
    # Condition for usage.
    # #
    if constants.LABEL_USAGE in evidences:
        returned_labels = recheck_returned_labels(matched_labels,
                                                  returned_labels,
                                                  constants.LABEL_USAGE)

        conditions.append("({})".format(" OR ".join([
            f"{constants.LABEL_USAGE}.individual LIKE '%, {x},%' + 'OR {constants.LABEL_USAGE}.individual LIKE '{x}, %' OR {constants.LABEL_USAGE}.individual LIKE '%, {x}'"
            for x in evidences[constants.LABEL_USAGE]
        ])))

    ##
    # Adjust returned labels
    # #
    adjusted_return_labels = []

    for return_label in returned_labels:
        return_label = f"{return_label.split('(')[1].split(':')[0]} as {(return_label.split(':')[1].split(')')[0]).title()}"

        adjusted_return_labels.append(return_label)

    return out.format(
        ', '.join(match_relation),
        ' \nAND '.join([f"{x}" for x in conditions]),
        ', '.join(adjusted_return_labels),
    )


In [17]:
def gen_query_ontology(text):
    table = prettytable.PrettyTable(["Step", "Result"])
    table.add_row(["Input", text])

    concepts = get_concepts(text)

    match_relation = get_match_relation(concepts.keys())

    evidences = get_entities(text)

    table.add_rows([["Match alias", concepts], ["Find individuals",
                                                evidences]])

    targets = list(set(concepts.keys()).difference(set(evidences.keys())))
    table.add_row(["Target concepts", targets])

    query = gen_query(concepts.keys(), evidences.keys(), targets, evidences,
                      match_relation)
    table.add_row(["Query", query])

    print(table)
    return query

In [36]:
# text = "nhà ở quận 1 thường có giá khoảng bao nhiêu"
# text = "nhà hoặc căn hộ giá khoảng 2 tỷ thì mua ở quận nào"
text = "mình đang có 2 tỷ, nên mua nhà hoặc căn hộ ở quận nào nhỉ"

cqlNodeQuery = gen_query_ontology(text)

+------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
|       Step       |                                                                           Result                                                                          |
+------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
|      Input       |                                                 mình đang có 2 tỷ, nên mua nhà hoặc căn hộ ở quận nào nhỉ                                                 |
|   Match alias    |                                  {'real_estate_type': 'căn hộ', 'transaction': 'mua', 'price': 'tỷ', 'district': 'quận'}                                  |
| Find individuals |                              {'price': [(2000000000.0, None)], 'transaction': ['mua'], 'real_e

In [25]:
# Connect neo4j DB.
driver = GraphDatabase.driver('bolt://localhost:7687',
                              auth=('neo4j', 'password'))


In [30]:
def run_query(query):
    with driver.session(database="htdb") as session:
        results = session.run(query)

        table_results = prettytable.PrettyTable(results.keys())
        for r in results:
            table_results.add_row(r.values())

        return table_results
        # return results


In [29]:
result_data = run_query(cqlNodeQuery)

print(result_data)